In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Training Results

In [ ]:
def makeResultDF(alg, num=0):
    retdf = None
    if alg == "dmc":
        path = "trained_models/uno_dmc_cuda_result/logs.csv"
        df = pd.read_csv(path)

        retdf = df["mean_episode_return_1"]
    else:
        numk = num*1000
        if num > 50:
            numk = f"{numk}_cuda"
        path = f"trained_models/uno_{alg}_{numk}_result/performance.csv"

        retdf = pd.read_csv(path)

        retdf.rename({"reward": alg.lower()}, axis=1, inplace=True)
    return retdf

In [ ]:
def plotDQNandNFSP(num):
    df_dqn = makeResultDF("dqn", num)
    df_nfsp = makeResultDF("nfsp", num)

    extracted_col = df_nfsp["nfsp"]
    df_combined = pd.concat([df_dqn, extracted_col], axis=1)

    plot = df_combined.plot(x='episode', kind='line', grid=True, figsize=(10, 6))
    plot.set_xlabel('episode', fontsize=16)
    plot.set_ylabel('reward', fontsize=16)

    file_name = f"training_dqn_nfsp"
    plot.get_figure().savefig(f"training_results/{file_name}.pdf")

In [ ]:
def plotDMC():
    path = "trained_models/uno_dmc_cuda_result/logs.csv"
    df = pd.read_csv(path)

    plot = df["mean_episode_return_1"].plot(kind='line', grid=True, figsize=(10, 6))
    plot.set_xlabel('time steps', fontsize=16)
    plot.set_ylabel('mean episode return', fontsize=16)

    file_name = f"training_dmc"
    plot.get_figure().savefig(f"training_results/{file_name}.pdf")

In [ ]:
def plotIndividual(alg, num):
    df = makeResultDF(alg, num)
    plot = df.plot(x='episode', kind='line', grid=True, legend=False, figsize=(10, 6))
    plot.set_xlabel('episode', fontsize=16)
    plot.set_ylabel('reward', fontsize=16)

    file_name = f"training_{alg}_{num}k"
    return [plot, file_name]

In [ ]:
plots = [
	plotIndividual("dqn", 50), # TODO: create again with correct data
	plotIndividual("dqn", 100),
	plotIndividual("dqn", 500),
	plotIndividual("nfsp", 50),
	plotIndividual("nfsp", 100),
	plotIndividual("nfsp", 500)
]

for plot, file_name in plots:
	plot.get_figure().savefig(f"training_results/{file_name}.pdf")

In [ ]:
plotDQNandNFSP(500)

In [ ]:
plotDMC()

# Evaluation results

In [ ]:
import re
import os

In [ ]:
def formatResult():
    directory = 'evaluate_results'

    algReg = r'(([A-Z]|[a-z]).*(?=(dqn|nfsp|dmc))|(?=_[a-z]).*\.pth)'
    ruleReg = r'(([A-Z]|[a-z]).*(?=rule)|(?<=rule).*v1)'

    finalResult = {}

    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        with open(f, 'r', encoding='utf16') as r:
            fileList = []
            tag = filename[::-1].replace('0000', 'k0')[::-1]
            for line in r:
                lineStr = re.sub(algReg, '', line) # alg
                lineStr = re.sub(ruleReg, '', lineStr) # rule

                data = lineStr.strip().split()
                fileList.append(data[1:])

            finalResult[tag] = fileList[1:]

    for key, value in finalResult.items():
        print(f'{key}: {value}')

    return finalResult

formatResult()

In [ ]:
df = pd.DataFrame([["DQN", 0.0232, 0.0324, 0.0556],
    ["NFSP", -0.0862,-0.1056, -0.0912], ["DMC", ]], columns=["Algotihm", "50k", "100k", "500k"])
# view data
print(df)

# plot grouped bar chart
main_pl = df.plot(x='Algotihm',
        kind='scatter',
        grid=True,
        title='Grouped Bar Graph with dataframe')

In [ ]:
# data from https://allisonhorst.github.io/palmerpenguins/

import matplotlib.pyplot as plt
import numpy as np

species = ("Adelie", "Chinstrap", "Gentoo")
penguin_means = {
    'Bill Depth': (18.35, 18.43, 14.98),
    'Bill Length': (38.79, 48.83, 47.50),
    'Flipper Length': (189.95, 195.82, 217.19),
}

x = np.arange(len(species))  # the label locations
width = 0.25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(layout='constrained')

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Length (mm)')
ax.set_title('Penguin attributes by species')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
ax.set_ylim(0, 250)

plt.show()